In [ ]:
# to automatically reload modules who's content has changed
%load_ext autoreload
%autoreload 2

# configure matplotlib
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

In [ ]:
from task_utils import *

In [ ]:
import funbo as fb
import funbo.plotting as fp

In [ ]:
domain_bounds = [('x', 0, 10)]
range_bounds = (-5, 5)

def make_objective(to_fit, sample_num, sample_dist):
    def objective(f):
        _, xmin, xmax = domain_bounds[0]
        # global reward
        R_g = fp.integrate(lambda x: (f(x) - to_fit(x))**2, (xmin, xmax))
        # local rewards
        R_ls = []
        if sample_dist == 'linear':
            reward_xs = np.linspace(xmin, xmax, num=sample_num)
        elif sample_dist == 'random':
            reward_xs = np.random.uniform(xmin, xmax, size=(sample_num,))
        else:
            raise ValueError()
        for x in reward_xs:
            R_l = (f(x)-to_fit(x))**2
            R_ls.append((x, R_l))
        return R_ls, R_g
    return objective

def to_fit(x):
    return np.sin(x) * 2*np.cos(x/4)

In [ ]:
class Coordinator(fb.Coordinator):
    def get_pre_phase_config(self, trial_num):
        c = fb.GPPriorSelectConfig(self.optimiser)
        return c

    def get_bayes_config(self, trial_num):
        c = fb.BayesSelectConfig(self.optimiser)
        c.extraction_config.aux_optimiser_params['exact_gradient'] = False
        c.extraction_config.tracking_l = 10
        return c

In [ ]:
np.random.seed(0)
objective = make_objective(to_fit, 10, 'linear')
opt = fb.Optimiser(objective, domain_bounds, range_bounds, desired_extremum='min')
opt.run(Coordinator(opt, 3, 5))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 8))
fp.plot_convergence(opt, best_R_g=0, ax=ax1)
fp.plot_consecutive_distance(opt, ax=ax2)
fp.plot_timings(opt, ax=ax3)
fig.tight_layout()

fp.plot_trials(opt, opt.trials, to_fit, color_by_reward=True)
fp.plot_surrogate_with_trials(opt, -1, to_fit)

inc_i, inc = opt.get_incumbent()
print('incumbent = trial {}'.format(inc_i))
fp.plot_trial_area(opt, inc, to_fit)